<a href="https://colab.research.google.com/github/rajy4683/EVA4S4/blob/master/S4EVA4_Attempt4_9941.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Final Accuracy: 99.41
###  Number of parameters - 10874

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
!pip install torchsummary
from torchsummary import summary

In [2]:
!pip install pytorch-ignite
!pip install torchsummary
!pip install wandb

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary

from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from torch.utils.data import DataLoader
from ignite.handlers import Checkpoint, DiskSaver


import logging
logging.propagate = False 
logging.getLogger().setLevel(logging.ERROR)

from argparse import ArgumentParser
from tqdm import tqdm
import os

In [0]:
import logging
logging.propagate = False 
logging.getLogger().setLevel(logging.ERROR)

In [5]:
import wandb
#wandb.init()
#!wandb login a6f947d2d2f69e7a8c8ca0f69811fd554f27d204
!wandb login a6f947d2d2f69e7a8c8ca0f69811fd554f27d204

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [0]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1) #input -? OUtput? RF
        self.conv1_bn = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)
        self.conv2_bn = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(16, 16, 3, padding=1)
        self.conv3_bn = nn.BatchNorm2d(16)
        self.conv4 = nn.Conv2d(16, 16, 3, padding=1)
        self.conv4_bn = nn.BatchNorm2d(16)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(16, 16, 3)
        self.conv5_bn = nn.BatchNorm2d(16)
        self.conv6 = nn.Conv2d(16, 16, 3)
        self.conv6_bn = nn.BatchNorm2d(16)

        self.conv7 = nn.Conv2d(16, 10, 1)

    def forward(self, x):
        # x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        # x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        # x = F.relu(self.conv6(F.relu(self.conv5(x))))
        # x = F.relu(self.conv7(x))
        # x = x.view(-1, 10)
        # return F.log_softmax(x)

        x = self.conv1_bn(F.relu(self.conv1(x))) ### Layer 1 ---->(Relu(BN(Conv)))
        x = self.pool1(self.conv2_bn(F.relu(self.conv2(x)))) ### Layer 2---> MaxPool((Relu(BN(Conv))))
        x = self.conv3_bn(F.relu(self.conv3(x))) ### Layer 3 ---->(Relu(BN(Conv)))
        x = self.pool2(self.conv4_bn(F.relu(self.conv4(x)))) ### Layer 4---> MaxPool((Relu(BN(Conv))))
        #########
        x = self.conv5_bn(F.relu(self.conv5(x))) ### Layer 5---> (Relu(BN(Conv)))        
        #x = F.adaptive_avg_pool2d(F.relu(self.conv6_bn(self.conv6(x))),(1,1))### Layer 6 ---> (Relu(BN(Conv))) F.adaptive_avg_pool2d(x, (1, 1))
        x = self.conv6_bn(F.relu(self.conv6(x)))
        x = F.adaptive_avg_pool2d(x, (1, 1))
        #F.avg_pool2d(x,x.size()[2:])### Layer 6 ---> (Relu(BN(Conv))) F.adaptive_avg_pool2d(x, (1, 1))
        x = self.conv7(x) #### Final Layer (Conv)
        x = x.view(-1, 10)


        # x = F.relu(self.conv1_bn(self.conv1(x))) ### Layer 1 ---->(Relu(BN(Conv)))
        # x = self.pool1(F.relu(self.conv2_bn(self.conv2(x)))) ### Layer 2---> MaxPool((Relu(BN(Conv))))
        # x = F.relu(self.conv3_bn(self.conv3(x))) ### Layer 3 ---->(Relu(BN(Conv)))
        # x = self.pool2(F.relu(self.conv4_bn(self.conv4(x)))) ### Layer 4---> MaxPool((Relu(BN(Conv))))
        # #########
        # x = F.relu(self.conv5_bn(self.conv5(x))) ### Layer 5---> (Relu(BN(Conv)))        
        # #x = F.adaptive_avg_pool2d(F.relu(self.conv6_bn(self.conv6(x))),(1,1))### Layer 6 ---> (Relu(BN(Conv))) F.adaptive_avg_pool2d(x, (1, 1))
        # x = F.relu(self.conv6_bn(self.conv6(x)))
        # x = F.adaptive_avg_pool2d(x, (1, 1))
        # #F.avg_pool2d(x,x.size()[2:])### Layer 6 ---> (Relu(BN(Conv))) F.adaptive_avg_pool2d(x, (1, 1))
        # x = self.conv7(x) #### Final Layer (Conv)
        # x = x.view(-1, 10)
        return F.log_softmax(x)

In [0]:
# class Net(nn.Module):
#     def __init__(self,start_channels=32, exponetate_layers=True):
#         super(Net, self).__init__()
#         self.start_channels = start_channels
#         self.multiplier = 2

#         if (exponetate_layers == False):
#             self.multiplier = 1
        
#         self.conv1 = nn.Conv2d(1, self.start_channels, 3, padding=1) #input -? OUtput? RF
#         self.conv1_bn = nn.BatchNorm2d(self.start_channels)
#         self.conv2 = nn.Conv2d(self.start_channels, self.start_channels*self.multiplier, 3, padding=1)
#         self.conv2_bn = nn.BatchNorm2d(self.start_channels*self.multiplier)

#         self.start_channels = self.start_channels * self.multiplier
#         self.pool1 = nn.MaxPool2d(2, 2)

#         self.conv3 = nn.Conv2d(self.start_channels, self.start_channels*self.multiplier, 3, padding=1)
#         self.conv3_bn = nn.BatchNorm2d(self.start_channels*self.multiplier)
#         #self.start_channels = self.start_channels * self.multiplier
#         self.conv4 = nn.Conv2d(self.start_channels, self.start_channels*self.multiplier, 3, padding=1)
#         self.conv4_bn = nn.BatchNorm2d(self.start_channels*self.multiplier)
#         #self.start_channels = self.start_channels * self.multiplier
#         self.pool2 = nn.MaxPool2d(2, 2)


#         # self.conv5 = nn.Conv2d(self.start_channels, self.start_channels*self.multiplier, 3)
#         # self.conv5_bn = nn.BatchNorm2d(self.start_channels*self.multiplier)
#         # #self.start_channels = self.start_channels * self.multiplier
#         # self.conv6 = nn.Conv2d(self.start_channels, self.start_channels*self.multiplier, 3)
#         # self.conv6_bn = nn.BatchNorm2d(self.start_channels*self.multiplier)
#         # #self.start_channels = self.start_channels * self.multiplier

#         # self.conv7 = nn.Conv2d(self.start_channels, 10, 1)

#     def forward(self, x):
#         x = F.relu(self.conv1_bn(self.conv1(x))) ### Layer 1 ---->(Relu(BN(Conv)))
#         x = self.pool1(F.relu(self.conv2_bn(self.conv2(x)))) ### Layer 2---> MaxPool((Relu(BN(Conv))))
#         x = F.relu(self.conv3_bn(self.conv3(x))) ### Layer 3 ---->(Relu(BN(Conv)))
#         x = self.pool2(F.relu(self.conv4_bn(self.conv4(x)))) ### Layer 4---> MaxPool((Relu(BN(Conv))))
#         #########
#         x = F.relu(self.conv5_bn(self.conv5(x))) ### Layer 5---> (Relu(BN(Conv)))        
#         #x = F.adaptive_avg_pool2d(F.relu(self.conv6_bn(self.conv6(x))),(1,1))### Layer 6 ---> (Relu(BN(Conv))) F.adaptive_avg_pool2d(x, (1, 1))
#         x = F.relu(self.conv6_bn(self.conv6(x)))
#         x = F.avg_pool2d(x,x.size()[2:])### Layer 6 ---> (Relu(BN(Conv))) F.adaptive_avg_pool2d(x, (1, 1))
#         x = self.conv7(x) #### Final Layer (Conv)
#         x = x.view(-1, 10)
#         return F.log_softmax(x)

In [0]:
# model = Net(start_channels=8,exponetate_layers=True).to(device)
# summary(model, input_size=(1, 28, 28))
# #wandb.watch(model, log="all")

TypeError: ignored

In [18]:
model = Net().to(device)
summary(model, input_size=(1, 28, 28))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3           [-1, 16, 28, 28]           1,168
       BatchNorm2d-4           [-1, 16, 28, 28]              32
         MaxPool2d-5           [-1, 16, 14, 14]               0
            Conv2d-6           [-1, 16, 14, 14]           2,320
       BatchNorm2d-7           [-1, 16, 14, 14]              32
            Conv2d-8           [-1, 16, 14, 14]           2,320
       BatchNorm2d-9           [-1, 16, 14, 14]              32
        MaxPool2d-10             [-1, 16, 7, 7]               0
           Conv2d-11             [-1, 16, 5, 5]           2,320
      BatchNorm2d-12             [-1, 16, 5, 5]              32
           Conv2d-13             [-1, 16, 3, 3]           2,320
      BatchNorm2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
train_loader.dataset.data.shape

torch.Size([60000, 28, 28])

In [0]:
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

from tqdm import tqdm
def train(args, model, device, train_loader, optimizer, epoch_number):
    model.train()
    pbar = tqdm(train_loader)
    train_loss = 0
    train_accuracy = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        train_accuracy += pred.eq(target.view_as(pred)).sum().item()

        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        train_loss += loss.item()

    train_loss /= len(train_loader.dataset)
    print('\nEpoch: {:.0f} Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        epoch_number, train_loss, train_accuracy, len(train_loader.dataset),
        100. * train_accuracy / len(train_loader.dataset)))
    
    wandb.log({        
        "Train Accuracy": 100. * train_accuracy / len(train_loader.dataset),
        "Train Loss": train_loss})


def test(args, model, device, test_loader,classes,epoch_number):
    model.eval()
    test_loss = 0
    correct = 0
    example_images = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
        #example_images.append(wandb.Image(
        #        data[0], caption="Pred: {} Truth: {}".format(classes[pred[0].item()], classes[target[0]])))

    test_loss /= len(test_loader.dataset)

    print('\nEpoch: {:.0f} Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        epoch_number, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    
    wandb.log({
        "Test Accuracy": 100. * correct / len(test_loader.dataset),
        "Test Loss": test_loss})


In [15]:
#model = Net(start_channels=16,exponetate_layers=False).to(device)
wandb.init(project="news4eva4")
wandb.watch_called = False # Re-run the model without restarting the runtime, unnecessary after our next release

# WandB – Config is a variable that holds and saves hyperparameters and inputs
config = wandb.config          # Initialize config
config.batch_size = 64          # input batch size for training (default: 64)
config.test_batch_size = 64    # input batch size for testing (default: 1000)
config.epochs = 20             # number of epochs to train (default: 10)
config.lr = 0.01               # learning rate (default: 0.01)
config.momentum = 0.9          # SGD momentum (default: 0.5) 
config.no_cuda = False         # disables CUDA training
config.seed = 1               # random seed (default: 42)
config.log_interval = 10     # how many batches to wait before logging training status

def main():
    use_cuda = not config.no_cuda and torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    
    # Set random seeds and deterministic pytorch for reproducibility
    # random.seed(config.seed)       # python random seed
    torch.manual_seed(config.seed) # pytorch random seed
    # numpy.random.seed(config.seed) # numpy random seed
    torch.backends.cudnn.deterministic = True

    # Load the dataset: We're training our CNN on CIFAR10 (https://www.cs.toronto.edu/~kriz/cifar.html)
    # First we define the tranformations to apply to our images
    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                        transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))
                        ])),
        batch_size=config.batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))
                        ])),
        batch_size=config.batch_size, shuffle=True, **kwargs)

    # Initialize our model, recursively go over all modules and convert their parameters and buffers to CUDA tensors (if device is set to cuda)
    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=config.lr,
                          momentum=config.momentum)
    
    # WandB – wandb.watch() automatically fetches all layer dimensions, gradients, model parameters and logs them automatically to your dashboard.
    # Using log="all" log histograms of parameter values in addition to gradients
    wandb.watch(model, log="all")

    for epoch in range(1, config.epochs + 1):
        train(config, model, device, train_loader, optimizer, epoch)
        test(config, model, device, test_loader, classes,epoch)
        
    # WandB – Save the model checkpoint. This automatically saves a file to the cloud and associates it with the current run.
    torch.save(model.state_dict(), "model.h5")
    wandb.save('model.h5')

if __name__ == '__main__':
    main()





# model = Net().to(device)
# summary(model, input_size=(1, 28, 28))

# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# wandb.watch(model, log="all")
# for epoch in range(1, 20):
#     train(model, device, train_loader, optimizer, epoch)
#     test(model, device, test_loader)

wandb: Wandb version 0.8.26 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
loss=0.06803488731384277 batch_id=937: 100%|██████████| 938/938 [00:17<00:00, 54.90it/s]



Epoch: 1 Train set: Average loss: 0.0031, Accuracy: 57167/60000 (95.278%)



  0%|          | 0/938 [00:00<?, ?it/s]


Epoch: 1 Test set: Average loss: 0.0472, Accuracy: 9871/10000 (98.710%)



loss=0.07141637802124023 batch_id=937: 100%|██████████| 938/938 [00:16<00:00, 55.93it/s]


...


Epoch: 19 Train set: Average loss: 0.0001, Accuracy: 59913/60000 (99.855%)



  0%|          | 0/938 [00:00<?, ?it/s]


Epoch: 19 Test set: Average loss: 0.0259, Accuracy: 9922/10000 (99.220%)



loss=0.0010610222816467285 batch_id=937: 100%|██████████| 938/938 [00:16<00:00, 56.47it/s]



Epoch: 20 Train set: Average loss: 0.0001, Accuracy: 59884/60000 (99.807%)


Epoch: 20 Test set: Average loss: 0.0216, Accuracy: 9928/10000 (99.280%)



In [0]:
torch.save(model.state_dict(), "model.h5")
wandb.save('model.h5')

['/content/wandb/run-20200206_101221-nhl168h8/model.h5']

In [17]:
from IPython.display import IFrame

IFrame(src='https://app.wandb.ai/rajy4683/news4eva4?workspace=user-rajy4683',width=700, height=600)